# 模型训练/调参/预测

In [2]:
from constant import *
from sklearn.metrics import roc_curve
from hyperopt import hp, tpe, fmin, Trials, STATUS_OK
import lightgbm as lgb
from data import loader,exporter
import numpy as np
import pandas as pd

In [23]:
def ks_stat(y_true, y_pred):
    """计算KS值的自定义评估函数"""
    fpr, tpr, _ = roc_curve(y_true, y_pred)  # 计算ROC曲线
    ks_value = np.max(np.abs(tpr - fpr))  # KS统计量
    return ks_value


def lgb_ks_eval(y_pred, dataset):
    """用于LightGBM的自定义KS评估函数"""
    y_true = dataset.get_label()
    ks_value = ks_stat(y_true, y_pred)
    # 返回 (名称, 计算的 KS 值, 是否越高越好)
    return 'ks', ks_value, True


# 自定义目标函数，用于贝叶斯优化
def objective(params, X, y, n_folds=5):
    """贝叶斯优化的目标函数，返回负的整体验证集KS分数"""
    # 设置模型参数
    params['boosting_type'] = 'gbdt'
    params['objective'] = 'binary'
    params['verbose'] = -1
    params['early_stopping_round'] = 20
    params['n_estimators'] = 10000

    kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=0)

    # 存储每一折的预测结果
    final_predictions = np.zeros(len(X))

    for train_idx, valid_idx in kf.split(X, y):
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

        # 建立LightGBM训练集
        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_valid = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

        # 训练LightGBM模型
        model = lgb.train(params,
                          lgb_train,
                          valid_sets=[lgb_train, lgb_valid],
                          feval=lgb_ks_eval,
                          callbacks=[lgb.early_stopping(stopping_rounds=50),
                                     lgb.log_evaluation(10)])

        # 验证集预测概率
        y_pred = model.predict(X_valid, num_iteration=model.best_iteration)

        # 将每一折的预测结果填入到相应的索引位置
        final_predictions[valid_idx] = y_pred

    # 计算整体的KS分数
    overall_score = ks_stat(y, final_predictions)

    # 返回负的KS分数作为最小化目标
    return {'loss': -overall_score, 'status': STATUS_OK}


# 超参数空间
param_space = {
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),  # 学习率
    'num_leaves': hp.choice('num_leaves', np.arange(20, 150, dtype=int)),  # 叶子数
    'max_depth': hp.choice('max_depth', np.arange(3, 12, dtype=int)),  # 树的最大深度
    'min_child_weight': hp.uniform('min_child_weight', 0.001, 10),  # 子叶节点的最小权重
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),  # 样本列采样率
    'scale_pos_weight': hp.uniform('scale_pos_weight', 1, 20),
    'subsample': hp.uniform('subsample', 0.5, 1.0),  # 样本采样率
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),  # L1正则化
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0)  # L2正则化
}


# 贝叶斯优化调参函数
def bayesian_optimize_lgbm(X, y, param_space, max_evals=50):
    trials = Trials()

    # 使用fmin函数进行贝叶斯优化
    best_params = fmin(
        fn=lambda params: objective(params, X, y),  # 优化目标
        space=param_space,  # 参数空间
        algo=tpe.suggest,  # 使用TPE算法
        max_evals=max_evals,  # 最大评估次数
        trials=trials  # 记录每次评估的结果
    )

    return best_params, trials

In [24]:
# 读取数据
df_target = loader.to_concat_df('TARGET')
df_flat = pd.read_csv(f'{dir_preprocess}/df_flat.csv')
df_flat = df_flat.merge(df_target, left_on=['CUST_NO', 'is_train'], right_on=['CUST_NO', 'is_train'], how='inner')
df_flat.drop(columns=['DATA_DAT', 'CARD_NO', 'CUST_NO'], inplace=True)

X = df_flat[df_flat['is_train'] == 1]
y = X.pop("FLAG")  # 目标标签列

# 执行贝叶斯优化
best_params, trials = bayesian_optimize_lgbm(X, y, param_space, max_evals=10)

print("Best parameters found:", best_params)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

Found `n_estimators` in params. Will use it instead of argument



Training until validation scores don't improve for 50 rounds
[10]	training's binary_logloss: 0.132047	training's ks: 0.823144	valid_1's binary_logloss: 0.18094	valid_1's ks: 0.519395
[20]	training's binary_logloss: 0.118563	training's ks: 0.870877	valid_1's binary_logloss: 0.182574	valid_1's ks: 0.535411
  0%|          | 0/10 [00:01<?, ?trial/s, best loss=?]

Found `n_estimators` in params. Will use it instead of argument



Training until validation scores don't improve for 50 rounds
[10]	training's binary_logloss: 0.132221	training's ks: 0.828503	valid_1's binary_logloss: 0.17836	valid_1's ks: 0.5334
[20]	training's binary_logloss: 0.116955	training's ks: 0.875865	valid_1's binary_logloss: 0.179604	valid_1's ks: 0.529141
  0%|          | 0/10 [00:03<?, ?trial/s, best loss=?]

...

Training until validation scores don't improve for 50 rounds                    
[10]	training's binary_logloss: 0.238627	training's ks: 0.634176	valid_1's binary_logloss: 0.257827	valid_1's ks: 0.518107
[20]	training's binary_logloss: 0.266602	training's ks: 0.678519	valid_1's binary_logloss: 0.295284	valid_1's ks: 0.514949
 90%|█████████ | 9/10 [01:28<00:11, 11.13s/trial, best loss: -0.542467198507184]

Found `n_estimators` in params. Will use it instead of argument



Training until validation scores don't improve for 50 rounds                    
[10]	training's binary_logloss: 0.240618	training's ks: 0.638914	valid_1's binary_logloss: 0.248452	valid_1's ks: 0.534923
[20]	training's binary_logloss: 0.26789	training's ks: 0.682527	valid_1's binary_logloss: 0.279088	valid_1's ks: 0.546545
 90%|█████████ | 9/10 [01:29<00:11, 11.13s/trial, best loss: -0.542467198507184]

Found `n_estimators` in params. Will use it instead of argument



Training until validation scores don't improve for 50 rounds                    
[10]	training's binary_logloss: 0.238619	training's ks: 0.63739	valid_1's binary_logloss: 0.254265	valid_1's ks: 0.5236
[20]	training's binary_logloss: 0.266104	training's ks: 0.683943	valid_1's binary_logloss: 0.288466	valid_1's ks: 0.540821
100%|██████████| 10/10 [01:31<00:00,  9.11s/trial, best loss: -0.542467198507184]
Best parameters found: {'colsample_bytree': 0.6983157656158572, 'learning_rate': 0.1705732511631721, 'max_depth': 4, 'min_child_weight': 9.775174703305787, 'num_leaves': 23, 'reg_alpha': 0.6014275790836761, 'reg_lambda': 0.30676387849859477, 'scale_pos_weight': 1.0099553852611183, 'subsample': 0.8925238455520927}


In [25]:
# 5折交叉验证
def lgb_5_fold(X, y, best_params, n_folds=5):
    best_params.update({
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'verbose': -1,
        'n_estimators': 10000,  # 和贝叶斯优化期间保持一致
        'early_stopping_round': 20
    })

    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=0)
    fold_scores = []
    final_predictions = np.zeros(len(X))  # 用于存储每一折的预测结果

    for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

        # 训练模型
        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_valid = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

        model = lgb.train(best_params,
                          lgb_train,
                          valid_sets=[lgb_train, lgb_valid],
                          feval=lgb_ks_eval,
                          callbacks=[lgb.early_stopping(stopping_rounds=50),
                                     lgb.log_evaluation(10)])

        # 保存模型
        model_path = f"{dir_model}/lgbm_fold_{fold + 1}.bin"
        model.save_model(model_path)
        print(f"Model for fold {fold + 1} saved at {model_path}")

        # 验证集预测
        y_pred = model.predict(X_valid, num_iteration=model.best_iteration)

        # 计算KS
        score = ks_stat(y_valid, y_pred)
        fold_scores.append(score)

        # 将每一折的预测结果保存到对应的索引位置
        final_predictions[valid_idx] = y_pred

        print(f"Fold {fold + 1} - Score: {score:.4f}")

    # 最终综合预测结果的评分
    overall_score = ks_stat(y, final_predictions)

    print(f"\nOverall Score: {overall_score:.4f}")

    return overall_score

In [26]:
# 执行5折交叉验证
best_score = lgb_5_fold(X, y, best_params)
print(f"Best score: {best_score}")


Found `n_estimators` in params. Will use it instead of argument


Training until validation scores don't improve for 50 rounds
[10]	training's binary_logloss: 0.157059	training's ks: 0.576152	valid_1's binary_logloss: 0.163124	valid_1's ks: 0.561215
[20]	training's binary_logloss: 0.149618	training's ks: 0.595574	valid_1's binary_logloss: 0.15999	valid_1's ks: 0.553792
[30]	training's binary_logloss: 0.145429	training's ks: 0.606795	valid_1's binary_logloss: 0.159353	valid_1's ks: 0.557227
Model for fold 1 saved at /Users/z/Data/Contest/2024/sh/dummy/model/lgbm_fold_1.bin
Fold 1 - Score: 0.5658


Found `n_estimators` in params. Will use it instead of argument


Training until validation scores don't improve for 50 rounds
[10]	training's binary_logloss: 0.156688	training's ks: 0.577266	valid_1's binary_logloss: 0.163487	valid_1's ks: 0.543723
[20]	training's binary_logloss: 0.14903	training's ks: 0.603021	valid_1's binary_logloss: 0.159818	valid_1's ks: 0.548952
[30]	training's binary_logloss: 0.144621	training's ks: 0.613624	valid_1's binary_logloss: 0.15864	valid_1's ks: 0.553868
[40]	training's binary_logloss: 0.140997	training's ks: 0.6388	valid_1's binary_logloss: 0.158075	valid_1's ks: 0.555845
[50]	training's binary_logloss: 0.138129	training's ks: 0.654463	valid_1's binary_logloss: 0.158351	valid_1's ks: 0.551328
Model for fold 2 saved at /Users/z/Data/Contest/2024/sh/dummy/model/lgbm_fold_2.bin
Fold 2 - Score: 0.5568


Found `n_estimators` in params. Will use it instead of argument


Training until validation scores don't improve for 50 rounds
[10]	training's binary_logloss: 0.156697	training's ks: 0.583152	valid_1's binary_logloss: 0.166882	valid_1's ks: 0.517907
[20]	training's binary_logloss: 0.149197	training's ks: 0.601976	valid_1's binary_logloss: 0.162898	valid_1's ks: 0.535064
[30]	training's binary_logloss: 0.145573	training's ks: 0.61526	valid_1's binary_logloss: 0.161948	valid_1's ks: 0.542605
[40]	training's binary_logloss: 0.142556	training's ks: 0.628371	valid_1's binary_logloss: 0.16142	valid_1's ks: 0.540941
[50]	training's binary_logloss: 0.139811	training's ks: 0.642123	valid_1's binary_logloss: 0.161107	valid_1's ks: 0.54724
[60]	training's binary_logloss: 0.136972	training's ks: 0.657321	valid_1's binary_logloss: 0.161233	valid_1's ks: 0.547164
[70]	training's binary_logloss: 0.134708	training's ks: 0.661718	valid_1's binary_logloss: 0.161718	valid_1's ks: 0.540082
Model for fold 3 saved at /Users/z/Data/Contest/2024/sh/dummy/model/lgbm_fold_3.b

Found `n_estimators` in params. Will use it instead of argument


Training until validation scores don't improve for 50 rounds
[10]	training's binary_logloss: 0.157008	training's ks: 0.574982	valid_1's binary_logloss: 0.163026	valid_1's ks: 0.543487
[20]	training's binary_logloss: 0.149493	training's ks: 0.597475	valid_1's binary_logloss: 0.159274	valid_1's ks: 0.552869
[30]	training's binary_logloss: 0.145258	training's ks: 0.609912	valid_1's binary_logloss: 0.158365	valid_1's ks: 0.557344
[40]	training's binary_logloss: 0.141428	training's ks: 0.628169	valid_1's binary_logloss: 0.158344	valid_1's ks: 0.562103
[50]	training's binary_logloss: 0.137799	training's ks: 0.639677	valid_1's binary_logloss: 0.15854	valid_1's ks: 0.562391
[60]	training's binary_logloss: 0.135215	training's ks: 0.654129	valid_1's binary_logloss: 0.158815	valid_1's ks: 0.556127
Model for fold 4 saved at /Users/z/Data/Contest/2024/sh/dummy/model/lgbm_fold_4.bin
Fold 4 - Score: 0.5665


Found `n_estimators` in params. Will use it instead of argument


Training until validation scores don't improve for 50 rounds
[10]	training's binary_logloss: 0.156165	training's ks: 0.567873	valid_1's binary_logloss: 0.166483	valid_1's ks: 0.531536
[20]	training's binary_logloss: 0.148325	training's ks: 0.595583	valid_1's binary_logloss: 0.162978	valid_1's ks: 0.530982
[30]	training's binary_logloss: 0.143799	training's ks: 0.613765	valid_1's binary_logloss: 0.162249	valid_1's ks: 0.535129
[40]	training's binary_logloss: 0.141151	training's ks: 0.621926	valid_1's binary_logloss: 0.162119	valid_1's ks: 0.540969
[50]	training's binary_logloss: 0.138316	training's ks: 0.634735	valid_1's binary_logloss: 0.162403	valid_1's ks: 0.537181
Model for fold 5 saved at /Users/z/Data/Contest/2024/sh/dummy/model/lgbm_fold_5.bin
Fold 5 - Score: 0.5428

Overall Score: 0.5485
Best score: 0.5485360295890676


In [22]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

def find_optimal_seed(X, y, param_space, n_folds=5, seed_list=[0, 42, 100, 2023]):
    best_seed = None
    lowest_std = float('inf')
    seed_scores = {}

    for seed in seed_list:
        skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)
        fold_scores = []

        for train_idx, valid_idx in skf.split(X, y):
            X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
            y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

            # 使用与之前一致的参数和模型训练方法
            model = lgb.LGBMClassifier(**param_space)  # 使用参数构建模型
            model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=50, verbose=False)
            
            y_pred = model.predict_proba(X_valid)[:, 1]
            ks = ks_stat(y_valid, y_pred)
            fold_scores.append(ks)

        # 计算该种子下的折间KS值的标准差
        std_ks = np.std(fold_scores)
        seed_scores[seed] = (np.mean(fold_scores), std_ks)

        if std_ks < lowest_std:
            best_seed = seed
            lowest_std = std_ks

    print(f"Best seed found: {best_seed} with lowest KS std: {lowest_std}")
    return best_seed, seed_scores


optimal_seed, seed_scores = find_optimal_seed(X, y, best_params, n_folds=5, seed_list=[0, 42, 100, 2023])

print(optimal_seed)
print(seed_scores)

'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
'early_stopping_rounds' argument is depr

Best seed found: 0 with lowest KS std: 0.009036162651410237
0
{0: (0.5519935649537354, 0.009036162651410237), 42: (0.5501777084193054, 0.029194183607442168), 100: (0.5534339207343374, 0.021856339442777126), 2023: (0.5484132032355762, 0.012979645226222662)}


In [27]:
# 测试集预测并生成结果文件
def predict_test_set(test_data, n_folds=5, output_file='upload.csv'):
    test_predictions = np.zeros(len(test_data))

    # 依次加载每一折的模型进行预测
    for fold in range(n_folds):
        model_path = f"{dir_model}/lgbm_dart_fold_{fold + 1}.bin"
        model = lgb.Booster(model_file=model_path)

        # 对测试集进行预测，并将每一折的预测加总
        test_predictions += model.predict(test_data)

    # 平均每一折的预测
    test_predictions /= n_folds

    # 生成预测文件
    submission = pd.DataFrame({'CUST_NO': test_data.index, 'PRED': test_predictions})
    submission.to_csv(output_file, index=False)
    print(f"Test predictions saved to {output_file}")